# Lý thuyết
**Lọc cộng tác lân cận** là phương pháp để hoàn thiện ma trận tiện ích, dự đoán đánh giá của người dùng lên sản phẩm, được chia làm 2 loại:
* **Lọc cộng tác lân cận dựa trên người dùng**: Đánh giá sự tương đồng của mỗi người dùng với nhau, nếu hai người dùng là tương đồng thì sản phẩm người dùng này yêu thích sẽ được gợi ý cho người dùng kia
* **Lọc cộng tác lân cận dựa trên sản phẩm**: Đánh giá sự tương đồng của mỗi sản phẩm với nhau, nếu hai sản phẩm là tương đồng và người dùng yêu thích một trong hai sản phẩm, hệ thống sẽ gợi tý sản phẩm còn lại cho người dùng đó

# Thực hành

## Lọc cộng tác lân cận dựa trên người dùng:
* Chuẩn hóa ma trận tiện ích:
    * Đối với mỗi người dùng:
        * Tính trung bình đánh giá của người dùng theo mọi sản phẩm
        * Cập nhật đánh giá mỗi sản phẩm = đánh giá cũ - trung bình
    * Tính ma trận tương tự trong đó phần tử ở hàng i cột j là độ tương đồng của người dùng i với người dùng j
        * Độ tương đồng cosine:
        $$sim(u_1, u_2) = cosine\_similarity(u_1, u_2) = \cos(u_1, u_2) = \frac{\displaystyle \textbf{u}_1^{T}\textbf{u}_2}{\left\|\textbf{u}_1\right\|_2^2 \left\|\textbf{u}_2\right\|_2^2}$$
    * Dự đoán độ quan tâm của người dùng u tới sản phẩm i:$$\hat{y}_{i,u} = \frac{\sum_{u_j\in N(u,i)}^{}\bar{y}_{i,u_j}sim(u,u_j)}{\sum_{u_j \in N(u,i)}^{}\left| sim(u,u_j) \right|}$$
      * Trong đó:
        * $ N(u,i) $ là tập hợp k người dùng tương tự với $u$ nhất mà đã đánh giá sản phẩm $i$
        * $\bar{y}_{i,u_j}$ là đánh giá đã được chuẩn hóa của người dùng $u_j$ đến sản phẩm $i$


In [28]:
from __future__ import print_function
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

### Xây dựng class NBCF để lọc cộng tác người dùng

In [29]:
class NBCF(object):
    def __init__(self, Y_data, k, sim_func= cosine_similarity):
        self.Y_data = Y_data        # Mỗi dòng của dữ liệu train là [user_id, item_id, rating]
        self.k = k                  # Số hàng xóm
        self.sim_func = sim_func    # Hàm đo độ tương đồng
        self.Ybar = None            # Chuẩn hóa dữ liệu sau này
        self.n_users = int(np.max(self.Y_data[:, 0])) + 1 # Số lượng người dùng tập train
        self.n_items = int(np.max(self.Y_data[:, 1])) + 1 # Số lượng phim tập train
    
    def fit(self):
        users = self.Y_data[:, 0]       # Lấy tất cả người dùng
        self.Ybar = self.Y_data.copy()  
        self.mean_user = np.zeros((self.n_users, ))
        for n in range(self.n_users):
            ids = np.where(users == n)[0].astype(np.int32) # Lấy tất cả những chỉ số hàng của đánh giá của người dùng n
            item_ids = self.Y_data[ids, 1]                 # Lấy tất cả những chỉ số phim mà người dùng đã đánh giá
            ratings = self.Y_data[ids, 2]                   # Lấy tất cả các đánh giá của người dùng
            self.mean_user[n] = np.mean(ratings) if ids.size > 0 else 0 # Tính đánh giá trung bình của người dùng
            self.Ybar[ids,2] = ratings - self.mean_user[n]              # Chuẩn hóa đánh giá của người dùng
        # Chuyển thành ma trân tiện ích, có hàng là items còn cột là users
        self.Ybar = sparse.coo_matrix((self.Ybar[:,2], 
                                       (self.Ybar[:,1], self.Ybar[:,0])),
                                       (self.n_items, self.n_users)).tocsr()
        # Tính ma trận tương tự
        self.S = self.sim_func(self.Ybar.T, self.Ybar.T)

    def pred(self, user, item):
        # Lấy chỉ số của những đánh giá cho item:
        ids = np.where(self.Y_data[:,1] == item)[0].astype(np.int32)
        # Lấy những users đã đánh giá item:
        users_rated_item = (self.Y_data[ids, 0]).astype(np.int32)
        # Độ tương đồng của user đầu vào với các user đã đánh giá item
        sim = self.S[user, users_rated_item]
        # Lấy ra k người dùng tương đồng nhất với user
        knns_ids = np.argsort(sim)[-self.k:]
        # Lấy ra độ tương đồng của người dùng với k người dùng đó
        knns_sim = sim[knns_ids]
        # Lấy ra đánh giá của k người dùng đó
        knns_rating = self.Ybar[item, users_rated_item[knns_ids]]
        # eps để tránh chia cho 0
        eps = 1e-8
        return (knns_rating * knns_sim).sum() / (np.abs(knns_sim).sum() + eps) + self.mean_user[user]


### Tiến hành lọc cộng tác người dùng với bộ dữ liệu MovieLens

#### Nhập dữ liệu

In [30]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
rating_base = pd.read_csv("ml-100k/ua.base", sep= '\t', names= r_cols)
rating_test = pd.read_csv("ml-100k/ua.test", sep= '\t', names= r_cols)

rate_train = rating_base.to_numpy()
rate_test = rating_test.to_numpy()

# Đánh số lại các chỉ số từ 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

#### Lọc công tác người dùng với k = 40 hàng xóm

In [31]:
rs = NBCF(rate_train, k = 40)
rs.fit()

n_tests = rate_test.shape[0]
SE = 0
for n in range(n_tests):
    pred = rs.pred(rate_test[n, 0], rate_test[n, 1])
    SE += (pred - rate_test[n, 2]) ** 2

RMSE = np.sqrt(SE / n_tests)

print(f"User - User CF RMSE: {RMSE}")

User - User CF RMSE: 0.9766365234453831


## Lọc cộng tác lân cận dựa trên sản phẩm
### Hạn chế của lọc cộng tác lân cận dựa trên người dùng:
* Khi số lượng người dùng lớn hơn số sản phẩm (thường xảy ra), việc đánh giá ma trận tương tự cho người dùng tốn rất nhiều chi phí hơn so với việc đánh giá ma trận tương tư cho sản phẩm.
* Người dùng thường xuyên không đánh giá sản phẩm nên ma trận tiện ích có dòng ứng với người dùng thường bị khuyết nhiều dẫn đến việc tính độ tương đồng giữa các người dùng khó đạt độ tin cậy cao. Tuy nhiên do số lượng người dùng nhiều hơn số lượng sản phẩm nên ma trận tiện ích có dòng ứng với sản phẩm ít bị khuyết thiếu hơn và việc tính độ tương đồng giữa các sản phẩm đạt được độ tin cậy cao hơn.
* Khi sử dụng NBCF dựa trên tương tự người dùng, do số đánh giá thường là ít nên mỗi khi người dùng đưa ra một đánh giá mới thì cần phải cập nhật đánh giá trung bình của người dùng ngay. Ngược lại nếu dựa trên sản phẩm, số người dùng đánh giá một sản phẩm lớn hơn nhiều nên việc có thêm một người dùng đánh giá sản phẩm có ảnh hưởng nhỏ hơn so với trường hợp trên

### Nguyên lý lọc cộng tác dựa trên sản phẩm

* Đối với lọc cộng tác dựa trên sản phẩm, ta so sánh độ tương đồng của sản phẩm bằng các đánh giá của người dùng lên sản phẩm đó. Hay nói cách khác, ta coi như sản phẩm "đánh giá" người dùng. Vì vậy chỉ cần chuyển vị ma trận tiện ích là có thể thực hiện lọc cộng tác dựa trên sản phẩm
  * Lấy vị dụ ở tập dữ liệu MovieLens100k, dữ liệu train cho NBCF tương đồng người dùng là [user_id, movie_id, rating] thì ta chỉ cần chuyển thành [movie_id, user_id, rating] là được

In [32]:
# Chuyển đổi dữ liệu
rate_train = rate_train[:, [1, 0, 2]]
rate_test = rate_test[:, [1, 0, 2]]

rs = NBCF(rate_train, k = 40)
rs.fit()

n_tests = rate_test.shape[0]
SE = 0
for n in range(n_tests):
    pred = rs.pred(rate_test[n, 0], rate_test[n, 1])
    SE += (pred - rate_test[n, 2]) ** 2

RMSE = np.sqrt(SE / n_tests)
print(f"Item - Item CF RMSE: {RMSE}")

Item - Item CF RMSE: 0.9688528283997285


# Kết luận
Lọc cộng tác sản phẩm cho kết quả chính xác hơn lọc cộng tác người dùng